In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('./drive/My Drive/')

In [0]:

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
#import emoji
import re
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack, csr_matrix, vstack
import pickle
#analyzer_emoji = SentimentIntensityAnalyzer()

In [0]:
# Preprocessing for Twitter Datas

'''Maybe nltk porter stemmer or punctuation can be try'''

df['text'] = df['text'].astype(str).fillna(' ')
# Lower case comment
df['text'] = df['text'].str.lower()
# Add num words of comment as feature
#df['num_words'] = df['text'].apply(lambda s: len(s.split()))
# Add num words unique of comment as feature
#df['num_unique_words'] = df['text'].apply(lambda s: len(set(w for w in s.split())))
# Add num words unique per num words of comment as feature
#df['words_vs_unique'] = df['num_unique_words'] / df['num_words'] * 100
# Add emojis features
df['text'] = df['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
print("Statistical features end!")

Statistical features end!


In [0]:
# Preprocessing for Twitter Datas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('turkish')

df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
data = pd.read_csv("hepsiburada.csv")

In [0]:
df = data.copy()
df.head()

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...


In [0]:
df['Rating'].unique().tolist()

[1, 0]

In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM,  Dropout
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
# get all labels and reviews as a list
target = df['Rating'].values.tolist()#negative=0, positive=1
data = df['Review'].values.tolist()#text data

In [0]:
seperation = int(len(data) * 0.80)
x_train, x_test = data[:seperation], data[seperation:]
y_train, y_test = target[:seperation], target[seperation:]

In [0]:

df.shape

(243497, 2)

In [0]:
# We will get 10000 most frequently used words in our dataset
num_words = 10000 #200000,100000,20000

# Define tokenizer with Keras...
# If we don't define num_words, then we use all words in our dataset.
tokenizer = Tokenizer(num_words=num_words)

In [0]:
# Now tokenize the data
tokenizer.fit_on_texts(data)

In [0]:
# saving tokenizer
import pickle

with open('turkish_tokenizer_000.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# load tokenizer
with open('turkish_tokenizer_000.pickle', 'rb') as handle:
    turkish_tokenizer = pickle.load(handle)

In [0]:
"""
Tokenize our training data. This will only works for the words in our 10000 tokenizer. 
If a word is not in 10000 tokenized words, it will be ignored.
"""
x_train_tokens = turkish_tokenizer.texts_to_sequences(x_train)

In [0]:
x_train[100]

'Bu fiyata bu kalite kaçırmayın derim '

In [0]:

x_train_tokens[100]

[5, 39, 5, 131, 323, 143]

In [0]:

x_test_tokens = turkish_tokenizer.texts_to_sequences(x_test)


In [0]:
"""
We will add Padding for our comments.
In RNN, we give predefined-sized inputs. But our comments consist of different sized inputs, so we need to define
a input size for comments. If size > comment, then add 0s for the gap, otherwise trim the comment.
"""

# How many tokens in each comment?
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]

# Convert list to numpy array
num_tokens = np.array(num_tokens)
num_tokens.shape

(243497,)

In [0]:
# In average, how many tokens in one comment?
np.mean(num_tokens)

20.744703220162876

In [0]:
# Max token amount?
np.max(num_tokens)

295

In [0]:
# Index of max token
np.argmax(num_tokens)

21941

In [0]:
# Define max tokens for all comments
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens) #returns float
max_tokens = int(max_tokens)
max_tokens

59

In [0]:
# How many tokens are smaller than max_tokens?
np.sum(num_tokens < max_tokens) / len(num_tokens) * 100  # output: 96%. which means we will only lose info in 4%.

95.97982726686571

In [0]:
# Let's add paddings... So, all datas will be in the same size.
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [0]:
# Check the sizes
print(x_train_pad.shape)
print(x_test_pad.shape)

(194797, 59)
(48700, 59)


In [0]:
# In Keras, we can get tokens from strings but not vice versa.
# So we will write a function to get strings from tokens
idx = turkish_tokenizer.word_index
#print(idx)
# in idx, key value pair like 'çok': 1. But we want it reverse.
inverse_map = dict(zip(idx.values(), idx.keys()))
first_five = {k: inverse_map[k] for k in sorted(inverse_map.keys())[:5]}
first_five

{1: 'çok', 2: 'bir', 3: 've', 4: 'ürün', 5: 'bu'}

In [0]:
def convert_tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token != 0]
    text = ' '.join(words)
    return text

In [0]:
model = Sequential() #define model


In [0]:
embedding_size = 50 # vector with 50 size for every word

In [0]:
"""
Now we will create an embedding layer in Keras.
We won't use word2vec or glove, instead we create word vectors randomly.
"""

# Add embedding layer to our model.
# embedding matris size = num_words * embedding_size -> 10.000 * 50
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [0]:
# 3-layered LSTM
model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=4, return_sequences=False))
model.add(Dropout(0.2))
# Dense layer: aka fully connected layer. Consists of one neuron.
model.add(Dense(1, activation='sigmoid'))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:

from tensorflow.python.keras.optimizers import Adam
# Adam optimizer
optimizer = Adam(lr=1e-3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
optimizer='adam',
metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 59, 16)            4288      
_________________________________________________________________
dropout_1 (Dropout)          (None, 59, 16)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 59, 8)             800       
_________________________________________________________________
dropout_2 (Dropout)          (None, 59, 8)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                

In [0]:
# epoch -> how many times we are going to train our data.
# batch_size -> feeding size
model.fit(x_train_pad, y_train, epochs=10, batch_size=256)




Epoch 1/10





194797/194797 [==============================] - 266s 1ms/step - loss: 0.2672 - acc: 0.9424
Epoch 2/10
194797/194797 [==============================] - 246s 1ms/step - loss: 0.1681 - acc: 0.9515
Epoch 3/10
194797/194797 [==============================] - 242s 1ms/step - loss: 0.1022 - acc: 0.9683
Epoch 4/10
194797/194797 [==============================] - 247s 1ms/step - loss: 0.0803 - acc: 0.9760
Epoch 5/10
194797/194797 [==============================] - 257s 1ms/step - loss: 0.0647 - acc: 0.9812
Epoch 6/10
194797/194797 [==============================] - 249s 1ms/step - loss: 0.0524 - acc: 0.9849
Epoch 7/10
194797/194797 [==============================] - 251s 1ms/step - loss: 0.0444 - acc: 0.9877
Epoch 8/10
194797/194797 [==============================] - 260s 1ms/step - loss: 0.0371 - acc: 0.9901
Epoch 9/10
194797/194797 [==============================] - 249s 1ms/step - loss: 0.0333 - acc: 0.9917
Epoch 10/10
194797/194797 [==============================] - 249s

In [0]:

result = model.evaluate(x_test_pad, y_test)
result

48700/48700 [==============================] - 171s 4ms/step


[0.24414843802609973, 0.9457905544147844]

In [0]:
accuracy = (result[1]) * 100
accuracy

94.57905544147845

In [0]:
text1 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text2 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem :("
text3 = "hiç resimde gösterildiği gibi değil..."
text4 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text5 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim. ürünü iade ediyorum"
text6 = "tam bir fiyat performans ürünü"
text7 = "güzel bir ürün değil"
texts = [text1, text2,text3,text4,text5,text6,text7]

In [0]:
tokens = turkish_tokenizer.texts_to_sequences(texts)

In [0]:
tokens = turkish_tokenizer.texts_to_sequences(texts)
tokens

[[104, 2, 1032, 2333, 1466, 5, 4, 5, 1779],
 [553, 61, 82, 27, 1, 458, 33, 3, 4, 9, 1031],
 [46, 1096, 6419, 20, 50],
 [177, 735, 7728, 82, 263, 105, 326, 16],
 [46, 5, 30, 177, 2, 1717, 1244, 19, 677, 83],
 [74, 2, 28, 111, 19],
 [7, 2, 4, 50]]

In [0]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)

In [0]:
model.predict(tokens_pad)

array([[0.00613517],
       [0.08107951],
       [0.00456542],
       [0.9993015 ],
       [0.0052653 ],
       [0.99948347],
       [0.96659845]], dtype=float32)

In [0]:

for i in model.predict(tokens_pad):
  if i < 0.5:
    print("negative")
  elif i >= 0.5:
    print("positive")


negative
negative
negative
positive
negative
positive
positive


In [0]:
def convert(texts):
    tokens = turkish_tokenizer.texts_to_sequences(texts)
    tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
    return model.predict(tokens_pad)[0][0]

In [0]:
from keras.models import load_model

model.save('my_Model.h5')  # creates a HDF5 file 'my_model.h5'
#del model  # deletes the existing model

# returns a compiled model
# identical to the previous one

#load model
#model = load_model('my_Model.h5')